In [1]:
import numpy as np 
import yfinance as yf 
import matplotlib.pyplot as plt 


In [6]:
# get data 
tickers = ["SPY", "IEF"]
start_date = "2023-01-01"
end_date = "2023-12-31"

prices = yf.download(tickers , start_date, end_date)["Close"]
prices 

[*********************100%***********************]  2 of 2 completed


Ticker,IEF,SPY
Date,,
2023-01-03,90.183556,370.367889
2023-01-04,90.874916,373.227203
2023-01-05,90.744125,368.967377
2023-01-06,91.911942,377.428589
2023-01-09,92.145500,377.214661
...,...,...
2023-12-22,92.551582,467.651276
2023-12-26,92.609337,469.625916
2023-12-27,93.330940,470.475098


In [8]:
returns = prices.pct_change().dropna( )
returns 

Ticker,IEF,SPY
Date,,
2023-01-04,0.007666,0.007720
2023-01-05,-0.001439,-0.011413
2023-01-06,0.012869,0.022932
2023-01-09,0.002541,-0.000567
2023-01-10,-0.006286,0.007013
...,...,...
2023-12-22,-0.000935,0.002010
2023-12-26,0.000624,0.004222
2023-12-27,0.007792,0.001808


In [9]:
# compute portfolio returns 
# Sum {weight* stock_return }
p_returns = 0.6 * returns["SPY"] + 0.4 *returns["IEF"]
p_returns 

Date
2023-01-04    0.007699
2023-01-05   -0.007424
2023-01-06    0.018907
2023-01-09    0.000676
2023-01-10    0.001693
                ...   
2023-12-22    0.000832
2023-12-26    0.002783
2023-12-27    0.004202
2023-12-28   -0.001299
2023-12-29   -0.002731
Length: 249, dtype: float64

In [14]:
# 1 year has 252 trading days
periods_per_year = 252
# count the number of days available in our data
n_periods = returns.shape[0]
annualized_return = np.exp(np.log(1+ p_returns).sum())**(periods_per_year/n_periods) - 1 
annualized_return*100

np.float64(17.123371083367367)

In [16]:
# risk free rate 
risk_free_rate = yf.download("^IRX",start_date, end_date)["Close"]
risk_free_rate

[*********************100%***********************]  1 of 1 completed


Ticker,^IRX
Date,
2023-01-03,4.255
2023-01-04,4.400
2023-01-05,4.498
2023-01-06,4.493
2023-01-09,4.483
...,...
2023-12-22,5.208
2023-12-26,5.203
2023-12-27,5.235


In [ ]:
# average risk free rate for the year as a fraction
avg_risk_free_rate = (risk_free_rate.mean()/100).values[0]
avg_risk_free_rate

np.float64(0.05045160009384155)

In [21]:
# downside returns 
downside_returns  = p_returns[p_returns <0 ]
# annualized downside volatility
downside_volatility = downside_returns.std() * 252**(1/2)
downside_volatility

np.float64(0.05273310798891019)

In [22]:
print(f"Annualized return: {annualized_return:.4f}")
print(f"Average risk free rate: {avg_risk_free_rate:.4f}")
print(f"Downside volatility: {downside_volatility:.4f} ")

Annualized return: 0.1712
Average risk free rate: 0.0505
Downside volatility: 0.0527 


# Sortino ratio

$$ SR = (r_p - r_f)/\sigma_d $$

$ \sigma_d $ is the downside volatility

In [24]:
sortino_ratio = (annualized_return-avg_risk_free_rate)/downside_volatility

In [26]:
print(f"Sortino ratio: {sortino_ratio:.4f} ")

Sortino ratio: 2.2904 
